# Pipeline on DAP for simulation results

In [ ]:
%set -J 4 -j 1
%run dap

[dap_1]
depends: Py_Module("feather"), Py_Module("pandas"), Py_Module("")

In [ ]:
[varbvs_1]
depends: Py_Module("pandasql"), Py_Module('fisher'), Py_Module('plotly')
parameter: sample_size = 2000
parameter: seed = [12]
parameter: prevalence = [0.01]
parameter: gamma = [(3,1), (5,1), (9,1)]
parameter: n_causal_gene = [100, 200]
parameter: name = 'data/del_sim'
input: for_each = ['gamma', 'prevalence', 'n_causal_gene', 'seed']
output: "${name}_%s.data.pkl" % "shape{}_scale{}_p{}_N{}_M{}_{}".format("${_gamma[0]}", "${_gamma[1]}", "${_prevalence}", int("${sample_size}") * 2, "${_n_causal_gene}", "${_seed}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from simulation import Environment, run_simulation 
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['n_causal_gene'] = ${_n_causal_gene}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, "shape{}_scale{}_p{}_N{}_M{}_{}".format(${_gamma[0]}, ${_gamma[1]}, ${_prevalence}, ${sample_size} * 2, ${_n_causal_gene}, ${_seed}))

In [ ]:
[varbvs_3]
depends: R_library("varbvs"), R_library("feather")
input: group_by = "single"
output: "${_input!nn}.varbvs.rds"
task:
R:
dat = as.matrix(feather::read_feather(${_input!r}))
# FIXME: need to check if the usage is correct
fit = varbvs::varbvs(dat[,-1], NULL, dat[,1], family = "binomial", logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
fit$summary = summary(fit)
fit$w = varbvs::normalizelogweights(fit$logw)
fit$pip = fit$alpha %*% c(fit$w)
fit$beta = fit$mu %*% c(fit$w)
fit$var_names = colnames(dat[-1])
saveRDS(fit, ${_output!r})